Step 1: Install Necessary Libraries
Make sure to install transformers, torch, and pyspellchecker:

In [ ]:
pip install pyspellchecker


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 12.2 MB/s eta 0:00:00


In [ ]:
pip install transformers torch pyspellchecker


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

**Step 2: Import Libraries**

In [ ]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
from spellchecker import SpellChecker


**Step 3: Load BERT Model and Tokenizer**

In [ ]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


**Step 4: Setup SpellChecker**

In [ ]:
# Initialize the SpellChecker
spell = SpellChecker()


**Step 5: Function to Detect and Correct Spelling Errors**

In [ ]:
def correct_spelling(text):
    # Split the input text into words
    words = text.split()
    corrected_words = []

    # Correct each word using SpellChecker
    for word in words:
        corrected_word = spell.correction(word)
        corrected_words.append(corrected_word)

    # Join corrected words into a string
    corrected_text = ' '.join(corrected_words)

    # Prepare input for BERT
    input_text = f"[CLS] {corrected_text} [SEP]"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Get BERT predictions
    with torch.no_grad():
        outputs = model(input_ids)
        predictions = outputs[0]

    # Decode the predictions
    predicted_tokens = tokenizer.convert_ids_to_tokens(torch.argmax(predictions, dim=2)[0])
    refined_text = tokenizer.convert_tokens_to_string(predicted_tokens)

    # Remove special tokens
    refined_text = refined_text.replace('[CLS] ', '').replace(' [SEP]', '')

    return refined_text


**Step 6: Testing the Spell Checker**

In [ ]:
text = "This is a smple sentense with some erors to test this nlp model "
corrected_text = correct_spelling(text)
print("Original text: ", text)
print("Corrected text: ", corrected_text)


Original text:  This is a smple sentense with some erors to test this nlp model 
Corrected text:  . . this is a simple sentence with some errors to test this . . . .
